# PostgreSQL Summary Stats and Window Functions

# Window Functions

When you are working with aggregate values it is necessary that use GROUP BY with **all** non-aggregate columns

Window functions are a class of functions that perform calculations on a result set that has already been generated, also referred to as a "window".
- You can use window functions to perform aggregate calculations without having to group your data, just as you did with a subquery in SELECT
- You can also use them to calculate information such as running totals, rankings, and moving averages.

## Key Differences
- window functions are processed after the entire query except the final ORDER BY statement. 
- the window function uses the result set to calculate information, as opposed to using the database directly. 
- window functions are available in PostgreSQL, Oracle, MySQL, but not in SQLite


Tem simplest application for window functions is numbering rows.
~~~~sql
SELECT
  *,
  -- Assign numbers to each row
  ROW_NUMBER() OVER() AS Row_N
FROM table_name
ORDER BY Row_N ASC;
~~~~

For instance, you can calculate the AVG of home_goal and away_goal, and follow it with the *OVER* clause. This clause tells SQL to *"pass this aggregate value over this existing result set."*
~~~~sql
SELECT date,    (home_goal + away_goal) AS goals,
       AVG(home_goal + away_goal) OVER() AS overall_avg
FROM match
WHERE season = '2011/2012';
~~~~

The OVER clause indicates that a function is a window function.

## ORDER BY
ORDER BY in OVER orders the rows related to the current row
- Example: Ordering by year in descending order in ROW_NUMBER's OVER clause will assign 1 to the most recent year's row
~~~~sql
SELECT year, event
       ROW_NUMBER() OVER(ORDER BY year DESC, event ASC) AS Row_N
FROM table_name
ORDER BY year, event;
~~~~
- ORDER inside OVER takes effect before the ORDER outside of it.

## LAG
LAG is a window function that takes a column and a number n and returns the column's value n rows before the current row. 
- Passing 1 as n returns the previous row's value.
- LAG(column, n) OVER (...)

## Window Partitions
PARTITION BY splits the table into partitions based on a column's unique values;

A partition allows you to calculate separate values for different categories established in a partition.
- calculate different calculations in the same column

~~~~sql
AVG(home_goal) OVER(PARTITION BY season)
~~~~

For instance, to answer how many goals were scored in each match, and how did that compare to season's average?

~~~~sql
SELECT date,
       (home_goal + away_goal) AS goals,
       AVG(home_goal + away_goal) OVER(PARTITION BY season) as season_avg
FROM match;
~~~~

### PARTITION by Multiple Columns
~~~~sql
AVG(home_goal) OVER(PARTITION BY season, name)
~~~~

PARTITION BY is a pretty straight forward addition to the OVER clause.
- You can partition calculations by 1 or more columns as necessary to answer a question you may have.
- You can use a PARTITION with any kind of window function
    - calculation, rank, or others

### The Four Functions
Relative
- LAG(column, n) return column's value at row n rows before the current row
- LEAD(column, n) return column's value at row n rows after the current row

Absolute
- FIRST_VALUE(column) returns the first value in the table or partition
- LAST_VALUE(column) returns the last value in the table or partition

For instance, 
For each year, fetch the current gold medalist and the gold medalist 2 competitions ahead of the current row.

~~~~sql
SELECT
  -- For each year, fetch the current and future medalists
  year,
  athlete AS current_champion,
  LEAD(athlete,2) OVER (ORDER BY year ASC) AS Future_Champion
FROM Discus_Medalists
ORDER BY Year ASC;
~~~~

## Rank Functions

- ROW_NUMBER() always assigns unique numbers, even if two rows' value are the same
- RANK() assings the same number to rows with identical values, skipping over the next number in such case
   - creates a column numbering your data set from highest to lowest, or lowest to highest
- DENSE_RANK() also assigns the same number to rows with identical values, but doesn't skip over the next numbers

~~~~sql
SELECT date,    (home_goal + away_goal) AS goals,
                RANK() OVER(ORDER BY home_goal + away_goal DESC) AS goals_rank
FROM match
WHERE season = '2011/2012';
~~~~

## Paging: Splitting data into (approximately) equal chunk
Uses
    - Many APIS return data in pages to reduce data being sent
    - Separating data into quartile or thirds to judge performance
Enter NTILE
    - NTILE(n) splits the data into n approximately equal pages

~~~~sql
WITH Events AS (
  SELECT DISTINCT Event
  FROM Summer_Medals)
  
SELECT
  --- Split up the distinct events into 111 unique groups
  DISTINCT event,
  NTILE(111) OVER (ORDER BY Event ASC) AS Page
FROM Events
ORDER BY Event ASC;
~~~~

## Aggregate Functions
~~~~sql
WITH Medals AS (...)
SELECT Year,
       Country,
       Medals,
       SUM(Meals) OVER (PARTITION BY Country ...)
FROM Medals;
~~~~

## Sliding windows/Frames
Sliding windows are functions that perform calculations relative to the current row of a data set.
- to calculate a wide variety of information that aggregates one row at a time down your data set
- can also be partitioned by one or more columns, just like a non-sliding window.
- contains specific functions within the OVER clause to specify the data you want to use in your calculations.

~~~~sql
ROWS BETWEEN <start> AND <finish>
~~~~ 
In the ROWS BETWEEN statement, you can specify a number of keywords as shown here. 

By default, a frame starts at the beginning of a table or partition and ends at the current row
- A frame always starts with RANGE BETWEEN or ROWS BETWEEN. 

- ROWS BETWEEN [START] AND [FINISH]
    - PRECEDING and FOLLOWING are used to specify the number of rows before, or after, the current row that you want to include in a calculation.
    - UNBOUNDED PRECEDING and UNBOUNDED FOLLOWING tell SQL that you want to include every row since the beginning, or the end, of the data set in your calculations.
    - CURRENT ROW tells SQL that you want to stop your calculation at the current row.
    
~~~~sql
WITH Scandinavian_Medals AS (
  SELECT
    Year, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('DEN', 'NOR', 'FIN', 'SWE', 'ISL')
    AND Medal = 'Gold'
  GROUP BY Year)

SELECT
  -- Select each year's medals
  year,
  medals,
  -- Get the max of the current and next years'  medals
  MAX(medals) OVER (ORDER BY year ASC
             ROWS BETWEEN CURRENT ROW
             AND 1 FOLLOWING) AS Max_Medals
FROM Scandinavian_Medals
ORDER BY Year ASC;
~~~~

### Moving Avarage
A moving average is the avarage of the last n periods
- used to indicate momentum/trends
- useful in eliminating seasonality

Moving total is the sum of last n periods
- used to indicate perfomance; if the sum is going down, overall perfomanc is going down
    
### ROWS vs RANGE
- RANGE BETWEEN treats duplicates in OVER's ORDER BY subclause as a single entity
- ROWS BETWEEN is almost always used over RANGE BETWEEN

# PIVOTING
Pivoting transforms a table by making columns out of the unique values of one of its columns.
- CROSSTAB allows you to pivot a table by a certain column. 
- You'll need to use the CREATE EXTENSION statement before using CROSSTAB
- CREATE EXTENSION makes extra functions in an extension available for use. 

~~~~sql
-- Create the correct extention to enable CROSSTAB
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
  SELECT
    Gender, Year, Country
  FROM Summer_Medals
  WHERE
    Year IN (2008, 2012)
    AND Medal = 'Gold'
    AND Event = 'Pole Vault'
  ORDER By Gender ASC, Year ASC;
-- Fill in the correct column names for the pivoted table
$$) AS ct (Gender VARCHAR,
           "2008" VARCHAR,
           "2012" VARCHAR)

ORDER BY Gender ASC;
~~~~

~~~~sql
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
  WITH Country_Awards AS (
    SELECT
      Country,
      Year,
      COUNT(*) AS Awards
    FROM Summer_Medals
    WHERE
      Country IN ('FRA', 'GBR', 'GER')
      AND Year IN (2004, 2008, 2012)
      AND Medal = 'Gold'
    GROUP BY Country, Year)

  SELECT
    Country,
    Year,
    RANK() OVER
      (PARTITION BY Year
       ORDER BY Awards DESC) :: INTEGER AS rank
  FROM Country_Awards
  ORDER BY Country ASC, Year ASC;
-- Fill in the correct column names for the pivoted table
$$) AS ct (Country VARCHAR,
           "2004" INTEGER,
           "2008" INTEGER,
           "2012" INTEGER)

Order by Country ASC;
~~~~

## ROLLUP and CUBE
ROLLUP is a GROUP BY subclause that includes extra rows for group-level aggregations. 
- GROUP BY Contry, ROLLUP(Medal) will count all Country and Medal level totals, then count only Country level totals and fill in Medal with nulls for tese rows.
- If you ROLLUP all GROUP BY columns, you'll have an additional row with the grand total.
- ROLLUP is hierarchical, de-aggergating from the leftmost provided column to the right-most.

CUBE is a non-hierarchical ROLLUP
- It generates all possible group-level aggregations


- Use ROLLUP when you have hierachical data (e.g., date parts) and don't want all possible group-level aggegations
- Use CUBE when you want all possible group-level aggegations

~~~~sql
-- Count the gold medals per country and gender
SELECT
  country,
  gender,
  COUNT(*) AS Gold_Awards
FROM Summer_Medals
WHERE
  Year = 2004
  AND Medal = 'Gold'
  AND Country IN ('DEN', 'NOR', 'SWE')
-- Generate Country-level subtotals
GROUP BY Country, ROLLUP(Gender)
ORDER BY Country ASC, Gender ASC;
~~~

# Other useful functions

## COALESCE
Takes a list of values and return the first non-null value, going from left to right
- Useful when using SQL operations that return nulls
~~~~sql
SELECT
  -- Replace the nulls in the columns with meaningful text
  COALESCE(Country, 'All countries') AS Country,
  COALESCE(Gender, 'All genders') AS Gender,
  COUNT(*) AS Awards
FROM Summer_Medals
WHERE
  Year = 2004
  AND Medal = 'Gold'
  AND Country IN ('DEN', 'NOR', 'SWE')
GROUP BY ROLLUP(Country, Gender)
ORDER BY Country ASC, Gender ASC;
~~~~


## STRING_AGG
Takes all the values of a column and concatenates them, with separator in between each value

~~~~sql
WITH Country_Medals AS (
  SELECT
    Country,
    COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE Year = 2000
    AND Medal = 'Gold'
  GROUP BY Country),

  Country_Ranks AS (
  SELECT
    Country,
    RANK() OVER (ORDER BY Medals DESC) AS Rank
  FROM Country_Medals
  ORDER BY Rank ASC)

-- Compress the countries column
SELECT STRING_AGG(Country, ', ')
FROM Country_Ranks
-- Select only the top three ranks
WHERE Rank <= 3;
~~~~